# POLSCI 3

## Week 11, Lecture Notebook 2: Analyzing experiments with multivariate regression

Earlier this week, I gave you a lot of warnings about using multivariate regression to study causality.

But there is one use of multivariate regression to study causality that is perfectly valid: using multivariate regression to analyze data from experiments! Once you get the hang of it, this turns out to be even easier than using our old `difference_in_means()` approach, and also allows us to do something else useful, too.

There are two main points in today's lecture.

## Today's first main point: How to interpret multivariate regressions when analyzing experiments

The first point is about how to interpret multivariate regressions when you're analyzing experiments.

Here, the concept of a "baseline category" is really important, and I'll show you an example using a new dataset, which we'll also use in class.

### Data for today

This dataset is from a pretty neat experiment that was conducted by some former graduate students at UC Berkeley (who once GSI'd for PS 3 several years ago!).

This data set is the result of a large-scale field experiment conducted in 25 cities across Germany, during which 3,797 unknowing bystanders were exposed to brief social encounters with confederates who revealed their ideas regarding gender roles.

There were a few aspects to the experiment. We'll cover only one of them in lecture today, and leave the other one for the in-class activity.

From the authors:

> Our intervention was set up to observe the behavior of unknowing experimental subjects (bystanders) who are exposed to a highly realistic and carefully choreographed sequence of social encounters in public spaces. The intervention followed four steps:
> 1. First, a female confederate approaches a bench at a train station where other individuals are waiting for their train and draws their attention by asking them a question (“Do you know if I can I buy tickets on the train?”). 
> 2. Shortly thereafter, and in the presence of the bystanders, the confederate receives a phone call (from one of the other confederates who was not acting in the specific iteration), and audibly converses with the caller in German...regarding a member of her family (her sister). The conversation is scripted in a manner that reveals the confederate's position on the women's right to choose to pursue a career versus having to stay home to take care of the family. **Note from Professor Broockman: this is the experimental manipulation we will describe below.**
> 3. At the end of the phone call, a bag that the confederate was holding seemingly tears, making her drop a number of lemons, which disperse on the train platform and the confederate appears to be in need of assistance to pick them up. 
> 4. In the final step, team members who were not a part of the intervention record whether each bystander helped the confederate retrieve her lemons. A collage of photographs that capture the key sequences of our experimental intervention are presented in Figure 1.
> 
> ![](scene.jpg)
> 
> _Figure 1: Unknowing bystanders watch and listen as the confederate takes a call and conducts a conversation with a friend (a), in the process revealing her attitudes toward the role of women in society (family and work). Following the phone call, the confederate drops her possessions (lemons), which disperse on the platform (b). We observe whether bystanders assist the confederate in collecting her possessions (c)._

Here were the contents of the phone calls:

> - In the regressive gender attitude condition, the confederate expresses disappointment with her sister, who has decided to get a job rather than stay at home and take care of her husband and kids. The confederate states that she believes her role as a woman is to stay at home and take care of her family.
> - In the progressive attitude condition, the confederate expresses her approval of her sister’s decision to get a job rather than stay home and take care of her husband and kids. She states that she believes that women should not sacrifice their careers to stay at home and take care of their family.
> - In the neutral control condition, the confederate has a conversation of roughly equal length about an innocuous matter unrelated to her attitudes regarding women and of no sociopolitical valence.
>
> The specific issue of women’s career advancement was chosen because it has been a crucial concern of the women’s rights movement in Germany; most—but not all—native women hold progressive views.

The authors also randomized whether the women were wearing hijabs. We'll ignore this for now -- we'll look at that during the in-class activity.

Let's take a look at the data:

In [ ]:
library(estimatr)

data <- read.csv("ps3_phonecall_clean_lecture.csv")
head(data)

Each row in the dataset is one interaction where a person dropped their lemons at a German bus stop after talking on the phone.

Here is a quick rundown of what each column means:

- `anayhelp`: outcome: whether any bystander offered help
- `genderatt`: "progresive, regressive, or neutral gender attitude (Progressive, Neutral, Regressive)
- `bystander`: number of bystanders 
- `femprop`: proportion of female bystanders

### Using regression to analyze experiments

Here's the values of that treatment variable again:

In [ ]:
table(data$genderatt)

Now, what happens if we run a regression, predicting the outcome (whether people get help) with that treatment variable?

In [ ]:
summary(lm(anyhelp ~ genderatt, data))

Notice that the treatment has three values but the regression is only showing you two coefficients. Why? Because **a regression on experimental data shows you the _difference_ between each of the treatment conditions and a baseline**. The baseline is the value of the treatment that isn't shown (we call this the "omitted category"). In this case, the Conservative condition isn't shown, so we know that the regression is showing us a) the effect of the Neutral condition _relative to the Conservative condition_ and the effect of the Progressive condition _relative to the Conservative condition_.

Moreover, the standard error, t-value, and p-value columns mean exactly what they meant before! (Unfortunately regression doesn't automatically export a confidence interval, you would have to calculate it by hand by using $\text{Confidence Interval} = \text{Estimate} +/- 1.96 * \text{Standard Error}$.)

In the language of `difference_in_means()`, the category regression doesn't show is the same as `condition1` (the baseline), and it can show us the difference between that condition and multiple others on each line.

Let's verify this is true by looking at this our old, old way.

In [ ]:
cons.call <- subset(data, genderatt == 'Conservative')
cons.mean <- mean(cons.call$anyhelp)
cons.mean

In [ ]:
neutral.call <- subset(data, genderatt == 'Neutral')
neutral.mean <- mean(neutral.call$anyhelp)
neutral.mean

In [ ]:
progressive.call <- subset(data, genderatt == 'Progressive')
progressive.mean <- mean(progressive.call$anyhelp)
progressive.mean

Do we see the same estimates as we did in the regression? Yes:

In [ ]:
neutral.mean - cons.mean

In [ ]:
progressive.mean - cons.mean

This is also identical to what we were doing with `difference_in_means()` before:

In [ ]:
difference_in_means(anyhelp ~ genderatt, data,
                    condition1 = 'Conservative', condition2 = 'Neutral')

In [ ]:
difference_in_means(anyhelp ~ genderatt, data,
                    condition1 = 'Conservative', condition2 = 'Progressive')

**Takeaway**: When you run a regression like `lm(outcome ~ treatment, data)`, you are comparing an *omitted category* of the treatment to all of the remaining categories, one-by-one. As long as there are no other variables in the regression except for the treatment variables, the `(Intercept)` gives you the mean of the outcome in this omitted group. The coefficients on each treatment tell you the difference in means between the mean in that omitted group and each of the other experimental groups.

------

## Today's second main point: When analyzing experimental data, control variables in regressions serve a different purpose: decreasing noise, not bias

We learned that experiments have no omitted variable bias, but they do have noise -- this is why we have standard errors and $p$-values, to quantify how much noise there is, and whether our estimate might have arisen by chance.

We also learned that, when analyzing observational data, the purpose of control variables is to try to reduce omitted variable bias (although it's unlikely this will ever fully succeed).

Today, I want to show you that _when analyzing experimental data, we can use control variables to reduce noise_!

How useful control variables will be depends on _how predictive the variables are of the outcome_. If a control variable predicts the outcome well, it gets rid of noise in the outcome that can't be due to the effect of the treatment. This leaves less noise left over.

Important note for your final projects: We cannot include anything as a covariate that treatment might itself affect, because this might explain some of the variation in the outcome that is due to the treatment. To be safe **only include as covariates variables that were measured prior to the treatment being implemented**.

We do this just like before, using the `+` sign to add addition variables.

We can start with the same regression as before:

In [ ]:
summary(lm(anyhelp ~ genderatt, data)) # same regression as before

Now let's add control variables:

In [ ]:
summary(lm(anyhelp ~ genderatt + bystander + femprop, data))

In this case, we can see that the covariates aren't very predictive of the outcome (they aren't even related in a statistically significant manner) and so the precision doesn't increase very much (in fact, not at all). However, there's no harm in including them.

**Key takeaways**:
- We can include both randomized treatments and non-randomized covariates in a regression. This is an optional enhancement that can help increase precision.
- Don't include covariates that were measured after treatment, since this could get rid of variation in the outcome related to the effect of the treatment's effects. We don't want that -- we only want to get rid of variation that couldn't possibly be due to the treatment (because it was measured prior).
- We still need to interpret the estimates (or "coefficients") on covariates as pure correlations and not as causal. Just because _a variable_ was randomized (the treatment) doesn't mean _every_ variable can now be interpreted as causal. For example, in this experiment, `genderatt` was randomized, but `femprop` wasn't! So, we can interpret the coefficients on the `genderatt` variables as causal, but not on the others (like `femprop`).